In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D

In [ ]:
#File importation
str0="ts_L60_Z12_A500_DX50_bias5_N10000"
fnamex ="DATA/x_" + str0 + ".csv"
fnamey ="DATA/y_" + str0 + ".csv"

x = np.loadtxt(fnamex,delimiter=',',dtype=float)
N = len(x)
print('Length of x = ',N)


categ = np.loadtxt(fnamey,delimiter=',', dtype=int)
#Number of categories
N_categ = 3
y = np.zeros((N,N_categ))

for n in range(N):
    y[n][categ[n]] = 1
    
print(y[0])



In [ ]:
#We saw that the nn was not converging and it might be
#due to some average, so we are going to remove it

xm = x.mean(axis=1)
for n in range(N):
    x[n] = x[n]-xm[n]
    
std = x.std(axis=1)
for n in range(N):
    x[n] = x[n] / std[n]  

In [ ]:
#Data visualization
plt.plot(x[0])
plt.plot(x[1])
plt.plot(x[2])
plt.show()

In [ ]:
#We asign the data for training and test
perc_train = 0.8
N_train = int(N * perc_train)
N_val = N-N_train

x_train = x[:N_train]
y_train = y[:N_train]

x_val = x[N_train:]
y_val = y[N_train:]

L =len(x[0])

print('Number of samples',N)
print('Training samples',N_train)
print('Validation samples',N_val)
print('Length of x[0]',L)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],L,1)
x_val = x_val.reshape(x_val.shape[0],L,1)
input_shape = (L,1)

In [ ]:
from keras import initializers,regularizers
np.random.seed(12345)

#We use l1 for Lazzo regularization
reg = regularizers.l1(0.1)   

#Now we initialize the NN weights
ini = initializers.RandomNormal(mean=0,stddev=0.05)

model = Sequential()
model.add(Conv1D(filters =5, kernel_size =11, 
                 kernel_regularizer = reg,
                 kernel_initializer = ini,
                 activation = 'relu',
                 input_shape =input_shape))

model.add(AveragePooling1D(5))
model.add(Conv1D(filters=5,kernel_size=7,activation="relu"))
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(N_categ,activation='softmax'))

print(model.summary())

from keras import optimizers
opt =optimizers.SGD(lr=0.01,momentum=0.9,nesterov=True,decay=1.e6)

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer='Adam',metrics=['accuracy'])

In [ ]:
batch_size =250
epochs = 30

fit = model.fit(x_train,y_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x_val,y_val),
               verbose=2, shuffle=True)

In [ ]:
for obs in ('accuracy','loss'):
    plt.figure(figsize=(6,4))
    plt.plot(fit.history[obs],'r',
            label =obs+' of training data')
    
    #dashed line!!
    plt.plot(fit.history['val_'+obs],'b--',
            label =obs+' of validation data')
    
    plt.ylabel(obs)
    plt.xlabel('epochs')
    plt.legend()
    plt.show()

3. Try another version where only one convolutional layer is
introduced instead of two, and where any number of
dense layers may be used, with the global constraint of
using a network with at most $600$ trainable parameters.

Is the number of parameters scaling more quickly by adding Dense layers or Conv1D
layers?

In [ ]:
name_key = lambda a, name: f'DATA/{a}_{name}.csv'
#File importation
def get_data(file_suffix):
    xfile = name_key('x',file_suffix)
    yfile = name_key('y',file_suffix)
    x = np.loadtxt(xfile,delimiter=',',dtype=float)
    y = np.loadtxt(yfile,delimiter=',',dtype=int)
    return x, y

def reshape_data(x,y, normalize = False):
    # Convert y
    N = len(x)
    N_cl = len(np.unique(y))
    _y = np.zeros((N,N_cl))
    for i in range(N): _y[i][y[i]] = 1
    
    perm = np.random.permutation(N)
    _x = x[perm]
    _y = _y[perm]

    if normalize:
        xm, std = _x.mean(axis=1), _x.std(axis=1)#should it be -1?
        for i in range(N): _x[i] = (_x[i]-xm[i])/std[i]
    
    return _x, _y



In [ ]:
# Get the data
str0="ts_comp_N10000"

x, y = get_data(str0)
x, y = reshape_data(x,y,True)

N = len(x)

# Balance the datasets
perc_train = 0.8
N_train = int(N * perc_train)
N_val = N-N_train

x_train = x[:N_train]
y_train = y[:N_train]

x_val = x[N_train:]
y_val = y[N_train:]

L =len(x[0])

print('Number of samples',N)
print('Training samples',N_train)
print('Validation samples',N_val)
print('Length of x[0]',L)

x_train = x_train.reshape(x_train.shape[0],L,1)
x_val = x_val.reshape(x_val.shape[0],L,1)
input_shape = (L,1)
output_size = len(y[0])


In [ ]:
# Build the model

def build_model(layers,output_size,output_activation = 'softmax'):
    model = Sequential()
    for layer in layers: model.add(layer)
    model.add(Dense(output_size,activation = output_activation))
    return model

layers = [Conv1D(filters =5, kernel_size =11, 
                 kernel_regularizer = reg,
                 kernel_initializer = ini,
                 activation = 'relu',
                 input_shape =input_shape),
          AveragePooling1D(5),
          Flatten(),
          Dense(6,activation='relu'),
          Dropout(0.2),
          Dense(12,activation='relu'),
          Dense(8,activation='relu')]

model_1 = build_model(layers, output_size)

from keras import optimizers
opt =optimizers.Adam()

model_1.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=opt,metrics=['accuracy'])

print(model_1.summary())


In [ ]:
batch_size =250
epochs = 300

fit = model_1.fit(x_train,y_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x_val,y_val),
               verbose=2, shuffle=True)

In [ ]:
model_1.predict(x_train)